# Example

In [ ]:
from __future__ import annotations

from itertools import repeat
from typing import TYPE_CHECKING

import geopandas as gpd
import numpy as np
import shapely

import tiny_retriever

if TYPE_CHECKING:
    from numpy.typing import NDArray
    from shapely import Point

In [ ]:
def query_catchments(points: NDArray[Point]) -> gpd.GeoDataFrame:
    """Query catchments that intersect with the given points."""
    points = np.asarray(points)
    if np.any(shapely.get_type_id(points) != 0):
        raise ValueError("Input must be a 1D array of Point objects.")

    url = "https://api.water.usgs.gov/geoserver/wmadata/ows"
    payload = {
        "service": "wfs",
        "version": "1.0.0",
        "request": "GetFeature",
        "typeName": "wmadata:catchmentsp",
        "outputFormat": "application/json",
        "srsName": "EPSG:4326",
    }
    pts_wkt = shapely.to_wkt(shapely.set_precision(shapely.force_2d(points), 1e-6))
    kwargs = ({"params": {**payload, "cql_filter": f"CONTAINS(the_geom, {p})"}} for p in pts_wkt)
    responses = tiny_retriever.fetch(repeat(url, len(points)), "text", request_kwargs=kwargs)
    gdf = gpd.GeoDataFrame.from_features(responses)
    shapely.prepare(points)
    c_idx, p_idx = shapely.STRtree(points).query(gdf.geometry, predicate="contains")
    return gdf.iloc[c_idx[np.argsort(p_idx)]]